In [45]:
import ode4
import controller
import ukf
import numpy as np
import os
import pinocchio as pin

DEBUG = True

Build a reduced model of the robot. If, when building the model of pinocchio from the Urdf file, we specify pin.JointModelFreeFlyer(), a special joint representing the floating base is added. This floating base adds 6 more degrees of freedom (nv + 6) that includes the translation in three dimensions (x, y, z) and the rotation in three dimensions (roll, pitch, yaw), that added to the 2 degrees of freedom of the wheels make up a total of 8 dofs.

In [48]:
# Initialize the robot model and data 
urdfFile = os.path.join('tiago.urdf')
full_model = pin.buildModelFromUrdf(urdfFile, pin.JointModelFreeFlyer())

# Base-related joints to keep
base_joints_to_keep = [
    'universe',             # Must be kept
    'root_joint',           # Floating base
    'wheel_left_joint',
    'wheel_right_joint',
]

# Get the IDs of all joints to keep and to lock (all joints except the ones to keep)
joints_to_keep_ids = [full_model.getJointId(joint) for joint in base_joints_to_keep if full_model.existJointName(joint)]
joints_to_lock_ids = [jid for jid in range(full_model.njoints) if jid not in joints_to_keep_ids]

# Set initial position for the joints we want to keep; the rest will be locked
initial_joint_config = np.zeros(full_model.nq)

# Build the reduced model
model = pin.buildReducedModel(full_model, joints_to_lock_ids, initial_joint_config)

# Check dimensions of the reduced model
print('Reduced model', model)

# Create the data for the reduced model
data = model.createData()

Reduced model Nb joints = 4 (nq=11,nv=8)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 wheel_left_joint: parent=1
  Joint 3 wheel_right_joint: parent=1



In [49]:
# Get the placement (position and orientation) of the root joint
root_joint_placement = data.oMi[model.getJointId('root_joint')]

# Print the placement
print('Base rotation:', root_joint_placement.rotation)
print('Base translation:', root_joint_placement.translation)

# Get the pose (position and orientation) of the root joint in (x, y, z, quat) format
root_joint_pose = pin.SE3ToXYZQUAT(root_joint_placement)  

# Print position and orientation
print("Position (roll, pitc, yaw):", root_joint_pose[:3])
print("Translation (x, y, z, s):", root_joint_pose[3:])

Base rotation: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Base translation: [0. 0. 0.]
Position (roll, pitc, yaw): [0. 0. 0.]
Translation (x, y, z, s): [0. 0. 0. 1.]


In [58]:
# Initialize robot configuration and velocity
q = pin.neutral(model)          # (x, y, z, quat)
v = pin.utils.zero(model.nv)

# Define the controller and UKF objects
contr = controller.Controller()
ukf_est = ukf.UKF()

# Starting configuration and initial state
starting_conf = np.zeros(model.nq)  # [x, y, theta, iL, iR, omegaLe, omegaRe]
#x0 = np.array(starting_conf + [0,0,0,0,0,0,0])  # Assuming initial velocities are zero

# Null space matrix and its derivative
#S = np.zeros((model.nv + len(starting_conf), 2))  # Adjust the size accordingly
#dS = np.zeros_like(S)

In [59]:
print(q)

[0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.]


In [ ]:
for joint in model.joints:
    print(joint)

JointModelRX
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1

JointModelFreeFlyer
  index: 1
  index q: 0
  index v: 0
  nq: 7
  nv: 6

JointModelRUBZ
  index: 2
  index q: 7
  index v: 6
  nq: 2
  nv: 1

JointModelRUBZ
  index: 3
  index q: 9
  index v: 7
  nq: 2
  nv: 1



In [ ]:
# Time step
dt = 0.1  # Example time step

# Loop for the simulation
for i in range(100):
    # Compute the torques
    tau = contr.command(starting_conf)
    if DEBUG: print("Torques", tau)

    # Update the state with the ode4 integration
    #x0 = ode4.ode4(model, data, x0, dt, tau, S, dS)
    #if DEBUG: print("ODE4", x0)
    
    # Update starting_conf for the next iteration
    #starting_conf = x0[:len(starting_conf)]
    
    #ukf.read_measures() #must be implemented
    #ukf.compute_sigma_points()

    if DEBUG: print("UKF", ukf_est.x)

    break

#contr.plot_results()
